In [ ]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import numpy as np
import time
import os
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

In [ ]:
#https://stackoverflow.com/questions/23267409/how-to-implement-retry-mechanism-into-python-requests-library

import logging
import requests

from requests.adapters import HTTPAdapter, Retry

logging.basicConfig(level=logging.DEBUG)

s = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

pwd = os.getcwd()
if 'L2 TVL' in pwd:
    prepend = ''
else:
    prepend = 'L2 TVL/'

In [ ]:
trailing_num_days = 7

start_date = date.today()-timedelta(days=trailing_num_days +1)

# start_date = datetime.strptime('2022-07-13', '%Y-%m-%d').date()



In [ ]:
#get all apps > 10 m tvl
all_api = 'https://api.llama.fi/protocols'
res = pd.DataFrame( r.get(all_api, headers=header).json() )
res = res[res['tvl'] > 10_000_000] ##greater than 10mil
print(len(res))
# print(res.columns)


In [ ]:

protocols = res[['slug','chainTvls']]
re = res['chainTvls']
# r[1].keys()
protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: list(x.keys()) )
protocols

In [ ]:
# protocols = protocols[ protocols['slug'] == 'uniswap-v3' ]
# api_str = 'https://api.llama.fi/protocol/'
# ad = pd.DataFrame( r.get(api_str).json()['chainTvls'] ).T[['tokens']]
# ad

In [ ]:
# api_str = 'https://api.llama.fi/protocol/uniswap'
# prot_req = r.get(api_str, headers=header).json()['chainTvls']
# # prot_req['Ethereum']
# prot_req

In [ ]:
#get by app
api_str = 'https://api.llama.fi/protocol/'
# print(protocols)
prod = []
for index,proto in protocols.iterrows():
#     print(proto)
    prot = proto['slug']
    chains = proto['chainTvls']
    apic = api_str + prot
#     time.sleep(0.1)
    try:
        prot_req = s.get(apic, headers=header).json()['chainTvls']
#         print(apic)
        for ch in chains:
            ad = pd.json_normalize( prot_req[ch]['tokens'] )
            ad_usd = pd.json_normalize( prot_req[ch]['tokensInUsd'] )
#             ad = ad.merge(how='left')
            if not ad.empty:
                ad = pd.melt(ad,id_vars = ['date'])
                ad = ad.rename(columns={'variable':'token','value':'token_value'})
                ad_usd = pd.melt(ad_usd,id_vars = ['date'])
                ad_usd = ad_usd.rename(columns={'variable':'token','value':'usd_value'})
                ad = ad.merge(ad_usd,on=['date','token'])
                
                ad['date'] = pd.to_datetime(ad['date'], unit ='s') #convert to days
                ad['token'] = ad['token'].str.replace('tokens.','', regex=False)
                ad['protocol'] = prot
                ad['chain'] = ch
        #         ad['start_date'] = pd.to_datetime(prot[1])
                # ad['date'] = ad['date'] - timedelta(days=1) #change to eod vs sod
                prod.append(ad)
    except:
        print('err')


In [ ]:
df_df_all = pd.concat(prod)
# df_df_all
print("done api")

In [ ]:
#filter down a bit so we can do trailing comp w/o doing every row
df_df = df_df_all[df_df_all['date'].dt.date >= start_date-timedelta(days=1) ]

#trailing comp
df_df['last_token_value'] = df_df.groupby(['token','protocol','chain'])['token_value'].shift(1)
#now actually filter
df_df = df_df[df_df['date'].dt.date >= start_date ]


In [ ]:

data_df = df_df.copy()
data_df['token_value'] = data_df['token_value'].replace(0, np.nan)
# price = usd value / num tokens
data_df['price_usd'] = data_df['usd_value']/data_df['token_value']

data_df.sort_values(by='date',inplace=True)

# net token change
data_df['net_token_flow'] = data_df['token_value'] - data_df['last_token_value']
# net token change * current price
data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']


data_df = data_df[abs(data_df['net_dollar_flow']) < 50_000_000_000] #50 bil error bar for bad prices
data_df = data_df[~data_df['net_dollar_flow'].isna()]


In [ ]:
netdf_df = data_df[['date','protocol','chain','net_dollar_flow','usd_value']]
netdf_df = netdf_df.fillna(0)
netdf_df = netdf_df.groupby(['date','protocol','chain']).sum(['net_dollar_flow','usd_value']) ##agg by app

#usd_value is the TVL on a given day
netdf_df = netdf_df.groupby(['date','protocol','chain','usd_value']).sum(['net_dollar_flow'])
netdf_df.reset_index(inplace=True)

netdf_df['cumul_net_dollar_flow'] = netdf_df[['protocol','chain','net_dollar_flow']]\
                                    .groupby(['protocol','chain']).cumsum()
netdf_df.reset_index(inplace=True)
netdf_df.drop(columns=['index'],inplace=True)


In [ ]:
#get latest
netdf_df['rank_desc'] = netdf_df.groupby(['protocol', 'chain'])['date'].\
                            rank(method='dense',ascending=False).astype(int)

In [ ]:
summary_df = netdf_df[  ( netdf_df['rank_desc'] == 1 ) &\
                        (~netdf_df['chain'].str.contains('-borrowed')) &\
                        (~netdf_df['chain'].str.contains('-staking')) &\
                        (~netdf_df['chain'].str.contains('-pool2')) &\
                        (~( netdf_df['chain'] == 'treasury') ) &\
                        (~( netdf_df['chain'] == 'borrowed') ) &\
                        (~( netdf_df['chain'] == 'staking') )
#                         & (~( netdf_df['chain'] == 'Ethereum') )
                        ]
summary_df = summary_df.sort_values(by='cumul_net_dollar_flow',ascending=False)
summary_df['pct_of_tvl'] = 100* summary_df['net_dollar_flow'] / summary_df['usd_value']
summary_df['flow_direction'] = np.where(summary_df['cumul_net_dollar_flow']>=0,1,-1)
summary_df['abs_cumul_net_dollar_flow'] = abs(summary_df['cumul_net_dollar_flow'])
# display(summary_df)
fig = px.treemap(summary_df[summary_df['abs_cumul_net_dollar_flow'] !=0], \
                 path=[px.Constant("all"), 'chain', 'protocol'], \
#                  path=[px.Constant("all"), 'token', 'chain', 'protocol'], \
                 values='abs_cumul_net_dollar_flow', color='flow_direction'
#                 ,color_discrete_map={'-1':'red', '1':'green'})
                ,color_continuous_scale='Spectral'
                )
# fig.data[0].textinfo = 'label+text+value'
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
# fig.update_layout(tickprefix = '$')

# fig.show()

In [ ]:
fig.write_image(prepend + "img_outputs/svg/net_app_flows.svg") #prepend + 
fig.write_image(prepend + "img_outputs/png/net_app_flows.png") #prepend + 
fig.write_html(prepend + "img_outputs/net_app_flows.html", include_plotlyjs='cdn')

In [3]:
# ! jupyter nbconvert --to python total_app_net_flows.ipynb

[NbConvertApp] Converting notebook total_app_net_flows.ipynb to python
[NbConvertApp] Writing 6873 bytes to total_app_net_flows.py
